In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm
import altair as alt
alt.data_transformers.disable_max_rows()
from collections import deque
import warnings
warnings.filterwarnings('ignore')
from rich.console import Console
from rich.table import Table
import numpy as np
from scipy.fft import fft
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.signal import convolve
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pywt
import matplotlib.pyplot as plt
console = Console()
import sys

sys.path.append("..")
from segmentation_scripts.utils import read_csv_file, get_data_directory_path, generate_table
from segmentation_scripts.generate_token_frequency_signal_processing_analysis import process_tokens

In [2]:
data_directory_path = get_data_directory_path()
preidentified_periodicals_df = read_csv_file(os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", "periodical_metadata", "classified_preidentified_periodicals_with_full_metadata.csv"))

In [3]:
all_frequencies_df = pd.read_csv("../datasets/all_volume_features_and_frequencies.csv")
console.print(f"Processed {len(all_frequencies_df)} volume features and frequencies.", style="bright_green")

Processed 4271 volume features and frequencies.

In [4]:
missing_volumes = preidentified_periodicals_df[~preidentified_periodicals_df.htid.isin(all_frequencies_df.htid)]
console.print(f"Missing Volumes: {len(missing_volumes)}", style="bright_red")
missing_titles = missing_volumes.lowercase_periodical_name.unique().tolist()
console.print(f"Missing Periodical Titles: {missing_titles}", style="bright_red")

Missing Volumes: 0

Missing Periodical Titles: []

In [23]:
subset_preidentified_periodicals_df = preidentified_periodicals_df[(preidentified_periodicals_df['lowercase_periodical_name'].isin(['arab_observer_and_the_scribe'])) & (preidentified_periodicals_df.volume_directory.notna())]

individual_htid = subset_preidentified_periodicals_df[subset_preidentified_periodicals_df.htid.isin(all_frequencies_df.htid)].htid.unique()[10]
individual_publication_directory = subset_preidentified_periodicals_df[subset_preidentified_periodicals_df.htid == individual_htid].publication_directory.values[0]
individual_volume_directory = subset_preidentified_periodicals_df[subset_preidentified_periodicals_df.htid == individual_htid].volume_directory.values[0]
console.print(f"Individual HTID: {individual_htid}", style="bright_green")
console.print(f"Individual Publication Directory: {individual_publication_directory}", style="bright_green")
console.print(f"Individual Volume Directory: {individual_volume_directory}", style="bright_green")
subset_frequencies_df = all_frequencies_df[all_frequencies_df.htid == individual_htid]
console.print(f"Processed {len(subset_frequencies_df)} frequencies for {individual_htid}.", style="bright_green")

Individual HTID: uc1.l0073177743

Individual Publication Directory: ht_ef_datasets/arab_observer_and_the_scribe

Individual Volume Directory: uc1_l0073177743

Processed 1 frequencies for uc1.l0073177743.

In [26]:
full_combined_results_path = os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", individual_publication_directory, "volumes", individual_volume_directory, "wavelet_analysis", individual_volume_directory + "_combined_results.csv")
if os.path.exists(full_combined_results_path):
	full_combined_results_df = pd.read_csv(full_combined_results_path)
	full_combined_results_df['htid'] = individual_htid
	console.print(f"Loaded {len(full_combined_results_df)} combined results from {full_combined_results_path}.", style="bright_green")
else:
	console.print(f"Could not find {full_combined_results_path}.", style="bright_red")

subset_combined_results_path = os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", individual_publication_directory, "volumes", individual_volume_directory, "wavelet_analysis", individual_volume_directory + "_subset_combined_results.csv")
if os.path.exists(subset_combined_results_path):
	subset_combined_results_df = pd.read_csv(subset_combined_results_path)
	subset_combined_results_df['htid'] = individual_htid
	console.print(f"Loaded {len(subset_combined_results_df)} subset combined results from {subset_combined_results_path}.", style="bright_green")
else:
	console.print(f"Could not find {subset_combined_results_path}.", style="bright_red")

wavelet_volume_data_path = os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", individual_publication_directory, "volumes", individual_volume_directory, "wavelet_analysis", individual_volume_directory + "_wavelet_volume_results.csv")
if os.path.exists(wavelet_volume_data_path):
	wavelet_volume_data_df = pd.read_csv(wavelet_volume_data_path)
	console.print(f"Loaded {len(wavelet_volume_data_df)} wavelet volume data from {wavelet_volume_data_path}.", style="bright_green")
else:
	console.print(f"Could not find {wavelet_volume_data_path}.", style="bright_red")

Loaded 275 combined results from 
/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_dat
asets/arab_observer_and_the_scribe/volumes/uc1_l0073177743/wavelet_analysis/uc1_l0073177743_combined_results.csv.

Loaded 136 subset combined results from 
/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_dat
asets/arab_observer_and_the_scribe/volumes/uc1_l0073177743/wavelet_analysis/uc1_l0073177743_subset_combined_results
.csv.

Loaded 1 wavelet volume data from 
/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_dat
asets/arab_observer_and_the_scribe/volumes/uc1_l0073177743/wavelet_analysis/uc1_l0073177743_wavelet_volume_results.
csv.

In [38]:
shared_cols = set(subset_combined_results_df.columns).intersection(set(wavelet_volume_data_df.columns))
avoid_cols = [col for col in wavelet_volume_data_df.columns if not col in shared_cols]
final_cols = avoid_cols + ['htid']
subset_combined_results_df = subset_combined_results_df.merge(wavelet_volume_data_df[final_cols], on='htid', how='left')
subset_combined_results_df['wavelet_family'] = subset_combined_results_df['wavelet'].str.extract(r'([a-zA-Z]+)')

subset_combined_results_df['wavelet_family'].value_counts()

wavelet_family
db      38
rbio    33
sym     19
bior    17
coif    17
gaus     8
haar     1
dmey     1
morl     1
mexh     1
Name: count, dtype: int64

In [40]:
melted_subset_combined_results_df = pd.melt(subset_combined_results_df, id_vars=['htid',  'wavelet', 'wavelet_type', 'signal_type', ], value_vars=['wavelet_rank', 'final_wavelet_rank', 'combined_wavelet_rank', 'combined_final_wavelet_rank'], var_name='rank_type', value_name='rank_value')

selection = alt.selection_multi(fields=['wavelet'], bind='legend')
sort_order = ['wavelet_rank', 'final_wavelet_rank', 'combined_wavelet_rank', 'combined_final_wavelet_rank']
alt.Chart(melted_subset_combined_results_df).mark_line(point=True).encode(
	x=alt.X('rank_type', sort=sort_order),
	y=alt.Y('rank_value', scale=alt.Scale(reverse=True)),  # Invert the y-axis
	color=alt.Color('wavelet', scale=alt.Scale(scheme='plasma'), legend=alt.Legend(symbolLimit=0, columns=8)),
	column='signal_type',
	row='wavelet_type',
	tooltip=['wavelet', 'rank_value', 'rank_type', 'signal_type', 'wavelet_type', 'htid'],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
	width=400,
	height=200
)

alt.Chart(...)

In [41]:
def calculate_rank_stability(df, rank_columns):
    """
    Calculate a stability metric for wavelet rankings based on multiple ranking columns.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing rank columns to evaluate.
    rank_columns : list of str
        Columns representing ranks to compare for stability.
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with an added 'rank_stability' column.
    """
    # Compute absolute differences between ranks
    for i, col_a in enumerate(rank_columns):
        for col_b in rank_columns[i+1:]:
            diff_col_name = f"{col_a}_vs_{col_b}_abs_diff"
            df[diff_col_name] = (df[col_a] - df[col_b]).abs()
    
    # Calculate the standard deviation of ranks across rank columns
    df['rank_std_dev'] = df[rank_columns].std(axis=1)
    
    # Normalize by the maximum possible rank
    max_rank = df[rank_columns].max().max()
    df['rank_stability'] = 1 - (df['rank_std_dev'] / max_rank)
    
    return df

# Example Usage
rank_columns = ['wavelet_rank', 'final_wavelet_rank', 'combined_wavelet_rank', 'combined_final_wavelet_rank']
subset_combined_results_df = calculate_rank_stability(subset_combined_results_df, rank_columns)

# Sort by rank stability
subset_combined_results_df.sort_values(by='rank_stability', ascending=False)[[ 'htid',  'wavelet', 'wavelet_type', 'signal_type', 'rank_stability', 'combined_final_wavelet_rank']]

,htid,wavelet,wavelet_type,signal_type,rank_stability,combined_final_wavelet_rank
23,uc1.l0073177743,coif17,DWT,raw,0.996642,24
24,uc1.l0073177743,bior3.9,DWT,raw,0.996642,25
13,uc1.l0073177743,bior3.9,DWT,raw,0.996031,14
14,uc1.l0073177743,coif16,DWT,raw,0.994199,15
26,uc1.l0073177743,bior3.7,DWT,raw,0.993894,27
...,...,...,...,...,...,...
121,uc1.l0073177743,sym18,DWT,smoothed,0.576601,122
122,uc1.l0073177743,sym19,DWT,smoothed,0.572121,123
123,uc1.l0073177743,sym3,DWT,smoothed,0.567640,124
124,uc1.l0073177743,sym20,DWT,smoothed,0.560787,125


In [42]:
family_summary = subset_combined_results_df.groupby(['signal_type', 'wavelet_family']).agg({
    'combined_final_score': ['mean', 'std', 'min', 'max'],
	'combined_final_wavelet_rank': ['mean', 'std', 'min', 'max', 'sum'],
    'rank_stability': ['mean', 'std'],
	'htid': ['count', 'nunique']
}).reset_index()

family_summary.columns = ['signal_type', 'wavelet_family', 'mean_combined_final_score', 'std_combined_final_score', 'min_combined_final_score', 'max_combined_final_score', 'mean_combined_final_wavelet_rank', 'std_combined_final_wavelet_rank', 'min_combined_final_wavelet_rank', 'max_combined_final_wavelet_rank', 'sum_combined_final_wavelet_rank', 'mean_rank_stability', 'std_rank_stability', 'count', 'unique_htid']
family_summary.sort_values(by=['unique_htid', 'sum_combined_final_wavelet_rank', 'mean_combined_final_wavelet_rank'], ascending=[False, True, True])[['signal_type', 'wavelet_family', 'mean_combined_final_wavelet_rank', 'sum_combined_final_wavelet_rank', 'count', 'mean_rank_stability', 'unique_htid']]

,signal_type,wavelet_family,mean_combined_final_wavelet_rank,sum_combined_final_wavelet_rank,count,mean_rank_stability,unique_htid
4,raw,haar,89.000000,89,1,0.964013,1
9,smoothed,dmey,126.000000,126,1,0.556306,1
6,raw,morl,127.000000,127,1,0.935599,1
5,raw,mexh,135.000000,135,1,0.934905,1
10,smoothed,gaus,133.500000,267,2,0.932399,1
11,smoothed,rbio,114.333333,343,3,0.618772,1
8,smoothed,bior,115.333333,346,3,0.618897,1
0,raw,bior,49.857143,698,14,0.979326,1
1,raw,coif,45.647059,776,17,0.964510,1
7,raw,rbio,26.066667,782,30,0.966357,1


In [47]:
limited_subset_combined_results_df = subset_combined_results_df[['htid', 'avg_tokens', 'avg_digits', 'total_pages', 'total_tokens', 'total_digits']].drop_duplicates()

limited_subset_combined_results_df[['avg_tokens', 'avg_digits', 'total_pages', 'total_tokens', 'total_digits']]


,avg_tokens,avg_digits,total_pages,total_tokens,total_digits
0,645.138859,6.691066,929,599334.0,6216


In [48]:
selection = alt.selection_point(fields=['wavelet'], bind='legend')
alt.Chart(subset_combined_results_df[['htid', 'wavelet', 'combined_final_wavelet_rank', 'wavelet_type', 'signal_type', 'wavelet_mode', 'wavelet_level', 'combined_final_score']]).mark_bar().encode(
	x=alt.X('combined_final_wavelet_rank:O', title='Wavelet Rank'),
	y=alt.Y('count()', title='Count'),
	color=alt.Color('wavelet:N', title='Wavelet Type', legend=alt.Legend(symbolLimit=0, columns=8)),
	row='wavelet_type:N',
	column='signal_type:N',
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
	tooltip=['htid', 'wavelet', 'signal_type', 'wavelet_type', 'combined_final_wavelet_rank', 'wavelet_mode', 'wavelet_level', 'combined_final_score']
).add_params(selection).properties(
	title='Wavelet Rank Distribution by Wavelength Type',
	width=600,
	height=200
).configure_legend(
	orient='bottom'
)

alt.Chart(...)

In [51]:
# Add rank bins to the data
subset_combined_results_df['rank_bin'] = pd.cut(
    subset_combined_results_df['combined_final_wavelet_rank'],
    bins=[0, 10, 20, 50, 100, subset_combined_results_df['combined_final_wavelet_rank'].max()],
    labels=['Top 10', 'Top 20', 'Top 50', 'Top 100', 'Beyond 100']
)

# Add unique htid count and stability metrics to the summary
rank_bin_summary = subset_combined_results_df.groupby(['wavelet_family', 'rank_bin']).agg(
    count=('combined_final_wavelet_rank', 'count'),
    unique_htid=('htid', 'nunique'),  # Count of unique volumes (htid)
    mean_rank_stability=('rank_stability', 'mean'),  # Mean rank stability
    std_rank_stability=('rank_stability', 'std')  # Standard deviation of rank stability
).reset_index()

# Total count for each rank_bin
bin_totals = rank_bin_summary.groupby('rank_bin')['count'].sum().reset_index()
bin_totals.rename(columns={'count': 'total_bin_count'}, inplace=True)
rank_bin_summary = rank_bin_summary.merge(bin_totals, on='rank_bin')

# Total unique htid per rank_bin
bin_totals_htid = rank_bin_summary.groupby('rank_bin')['unique_htid'].sum().reset_index()
bin_totals_htid.rename(columns={'unique_htid': 'total_bin_unique_htid'}, inplace=True)
rank_bin_summary = rank_bin_summary.merge(bin_totals_htid, on='rank_bin')

# Proportion of all wavelets in each bin (relative to total bin count)
rank_bin_summary['global_proportion'] = rank_bin_summary['count'] / rank_bin_summary['total_bin_count']

# Proportion of unique htid in each bin relative to total unique htid for that bin
rank_bin_summary['htid_proportion'] = rank_bin_summary['unique_htid'] / rank_bin_summary['total_bin_unique_htid']

# Sort order for consistent visualization
sort_order = ['Top 10', 'Top 20', 'Top 50', 'Top 100', 'Beyond 100']

selection = alt.selection_multi(fields=['wavelet_family'], bind='legend')
# Create a bar chart to include rank stability metrics
global_chart = alt.Chart(rank_bin_summary).mark_bar().encode(
    x=alt.X('rank_bin:N', title='Rank Bin', sort=sort_order),
    y=alt.Y('global_proportion:Q', title='Proportion of All Volumes (htid)', stack='normalize'),
    color=alt.Color('wavelet_family:N', title='Wavelet Family', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        'wavelet_family',
        'rank_bin',
        'count',
        'global_proportion',
        'unique_htid',
        'htid_proportion',
        'mean_rank_stability',
        'std_rank_stability'
    ],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin (All Volumes)',
    width=300,
    height=300
)

htid_chart = alt.Chart(rank_bin_summary).mark_bar().encode(
    x=alt.X('rank_bin:N', title='Rank Bin', sort=sort_order),
    y=alt.Y('htid_proportion:Q', title='Proportion of Unique Volumes (htid)', stack='normalize'),
    color=alt.Color('wavelet_family:N', title='Wavelet Family', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        'wavelet_family',
        'rank_bin',
        'count',
        'global_proportion',
        'unique_htid',
        'htid_proportion',
        'mean_rank_stability',
        'std_rank_stability'
    ],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin (Unique Volumes)',
    width=300,
    height=300
)

# Scatter plot for rank stability metrics
htid_global_chart = alt.Chart(rank_bin_summary).mark_point(filled=True).encode(
    x='htid_proportion:Q',
    y='global_proportion:Q',
    color='wavelet_family:N',
    tooltip=[
        'wavelet_family', 
        'rank_bin', 
        'count', 
        'global_proportion', 
        'unique_htid', 
        'htid_proportion', 
        'mean_rank_stability', 
        'std_rank_stability'
    ],
    shape=alt.Shape('rank_bin:N', sort=sort_order),
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin with Stability Metrics',
    width=300,
    height=300
)

stability_chart = alt.Chart(rank_bin_summary).mark_point(filled=True).encode(
	y='mean_rank_stability:Q',
	x='std_rank_stability:Q',
	color='wavelet_family:N',
	tooltip=[
		'wavelet_family', 
		'rank_bin',
		'count',
		'global_proportion',
		'unique_htid',
		'htid_proportion',
		'mean_rank_stability',
		'std_rank_stability'
	],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
	shape=alt.Shape('rank_bin:N', sort=sort_order)
).add_params(selection).properties(
	title='Rank Stability Metrics by Wavelet Family',
	width=300,
	height=300
)

# Combine charts
alt.vconcat(alt.hconcat(global_chart, htid_chart), alt.hconcat(htid_global_chart, stability_chart))

alt.VConcatChart(...)

In [56]:
rank_bin_summary = rank_bin_summary[rank_bin_summary.rank_bin == 'Top 10'].sort_values(by=['global_proportion', 'htid_proportion', 'mean_rank_stability', 'std_rank_stability', 'count', 'unique_htid'], ascending=[False, False, False, True, False, False])

rank_bin_summary

,wavelet_family,rank_bin,count,unique_htid,mean_rank_stability,std_rank_stability,total_bin_count,total_bin_unique_htid,global_proportion,htid_proportion
40,rbio,Top 10,10,1,0.970446,0.004305,10,1,1.0,1.0
0,bior,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
5,coif,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
10,db,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
15,dmey,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
20,gaus,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
25,haar,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
30,mexh,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
35,morl,Top 10,0,0,NaN,NaN,10,1,0.0,0.0
45,sym,Top 10,0,0,NaN,NaN,10,1,0.0,0.0


In [61]:
top_wavelet_family = rank_bin_summary.iloc[0].wavelet_family

subset_combined_results_df[subset_combined_results_df.wavelet_family == top_wavelet_family].sort_values(by=['combined_final_wavelet_rank', 'rank_stability'], ascending=True)[['htid', 'wavelet', 'wavelet_type', 'signal_type', 'wavelet_mode', 'wavelet_level', 'combined_final_wavelet_rank', 'combined_final_score', 'rank_stability']]

,htid,wavelet,wavelet_type,signal_type,wavelet_mode,wavelet_level,combined_final_wavelet_rank,combined_final_score,rank_stability
0,uc1.l0073177743,rbio3.9,DWT,raw,antireflect,1.0,1,0.691926,0.977101
1,uc1.l0073177743,rbio3.7,DWT,raw,smooth,1.0,2,0.662267,0.966110
2,uc1.l0073177743,rbio3.7,DWT,raw,zero,1.0,3,0.661835,0.970079
3,uc1.l0073177743,rbio3.7,DWT,raw,constant,1.0,4,0.661715,0.970079
4,uc1.l0073177743,rbio3.5,DWT,raw,periodic,1.0,5,0.660734,0.965194
5,uc1.l0073177743,rbio3.9,DWT,raw,reflect,1.0,6,0.660102,0.976796
6,uc1.l0073177743,rbio3.7,DWT,raw,antireflect,1.0,7,0.658429,0.972827
7,uc1.l0073177743,rbio3.5,DWT,raw,antireflect,1.0,8,0.658167,0.966721
8,uc1.l0073177743,rbio3.7,DWT,raw,antisymmetric,1.0,9,0.657561,0.972522
9,uc1.l0073177743,rbio3.5,DWT,raw,reflect,1.0,10,0.653857,0.967026


In [62]:
# Normalize rank and rank stability
subset_combined_results_df['normalized_rank'] = subset_combined_results_df['combined_final_wavelet_rank'] / subset_combined_results_df['combined_final_wavelet_rank'].max()
subset_combined_results_df['normalized_stability'] = 1 - subset_combined_results_df['rank_stability']  # Penalize instability

# Define weights for rank and stability
alpha = 0.5  # Weight for rank
beta = 0.5   # Weight for stability

# Compute composite score
subset_combined_results_df['composite_score'] = (
    alpha * subset_combined_results_df['normalized_rank'] + 
    beta * subset_combined_results_df['normalized_stability']
)

# Sort by composite score (ascending)
sorted_results = subset_combined_results_df.sort_values(by='composite_score', ascending=True)

# Select relevant columns to display
sorted_results[['htid', 'wavelet', 'wavelet_type', 'signal_type', 'wavelet_mode', 
                'wavelet_level', 'combined_final_wavelet_rank', 
                'combined_final_score', 'rank_stability', 'composite_score']]

,htid,wavelet,wavelet_type,signal_type,wavelet_mode,wavelet_level,combined_final_wavelet_rank,combined_final_score,rank_stability,composite_score
0,uc1.l0073177743,rbio3.9,DWT,raw,antireflect,1.0,1,6.919263e-01,0.977101,0.015126
1,uc1.l0073177743,rbio3.7,DWT,raw,smooth,1.0,2,6.622671e-01,0.966110,0.024298
2,uc1.l0073177743,rbio3.7,DWT,raw,zero,1.0,3,6.618354e-01,0.970079,0.025990
3,uc1.l0073177743,rbio3.7,DWT,raw,constant,1.0,4,6.617150e-01,0.970079,0.029666
5,uc1.l0073177743,rbio3.9,DWT,raw,reflect,1.0,6,6.601024e-01,0.976796,0.033661
...,...,...,...,...,...,...,...,...,...,...
121,uc1.l0073177743,sym18,DWT,smoothed,antireflect,1.0,122,-3.614631e+02,0.576601,0.660229
122,uc1.l0073177743,sym19,DWT,smoothed,antireflect,1.0,123,-4.886632e+02,0.572121,0.666145
123,uc1.l0073177743,sym3,DWT,smoothed,antireflect,1.0,124,-1.036124e+03,0.567640,0.672062
124,uc1.l0073177743,sym20,DWT,smoothed,antireflect,1.0,125,-1.625906e+03,0.560787,0.679165


In [92]:
subset_preidentified_periodicals_df = preidentified_periodicals_df[(preidentified_periodicals_df['lowercase_periodical_name'].isin(['arab_observer_and_the_scribe'])) & (preidentified_periodicals_df.volume_directory.notna())]

volume_dfs = []
for index, row in subset_preidentified_periodicals_df.iterrows():
	individual_htid = row.htid
	individual_publication_directory = row.publication_directory
	individual_volume_directory = row.volume_directory
	# console.print(f"Individual HTID: {individual_htid}", style="bright_green")
	# console.print(f"Individual Publication Directory: {individual_publication_directory}", style="bright_green")
	# console.print(f"Individual Volume Directory: {individual_volume_directory}", style="bright_green")
	subset_frequencies_df = all_frequencies_df[all_frequencies_df.htid == individual_htid]
	# console.print(f"Processed {len(subset_frequencies_df)} frequencies for {individual_htid}.", style="bright_green")

	

	subset_combined_results_path = os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", individual_publication_directory, "volumes", individual_volume_directory, "wavelet_analysis", individual_volume_directory + "_subset_combined_results.csv")
	if os.path.exists(subset_combined_results_path):
		subset_combined_results_df = pd.read_csv(subset_combined_results_path)
		subset_combined_results_df['htid'] = individual_htid
		# console.print(f"Loaded {len(subset_combined_results_df)} subset combined results from {subset_combined_results_path}.", style="bright_green")
	# else:
	# 	console.print(f"Could not find {subset_combined_results_path}.", style="bright_red")

	wavelet_volume_data_path = os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", individual_publication_directory, "volumes", individual_volume_directory, "wavelet_analysis", individual_volume_directory + "_wavelet_volume_results.csv")
	if os.path.exists(wavelet_volume_data_path):
		wavelet_volume_data_df = pd.read_csv(wavelet_volume_data_path)
	# 	console.print(f"Loaded {len(wavelet_volume_data_df)} wavelet volume data from {wavelet_volume_data_path}.", style="bright_green")
	# else:
	# 	console.print(f"Could not find {wavelet_volume_data_path}.", style="bright_red")
	
	if not wavelet_volume_data_df.empty and not subset_combined_results_df.empty:
		shared_cols = set(subset_combined_results_df.columns).intersection(set(wavelet_volume_data_df.columns))
		avoid_cols = [col for col in wavelet_volume_data_df.columns if not col in shared_cols]
		final_cols = avoid_cols + ['htid']
		subset_combined_results_df = subset_combined_results_df.merge(wavelet_volume_data_df[final_cols], on='htid', how='left')
		subset_combined_results_df['wavelet_family'] = subset_combined_results_df['wavelet'].str.extract(r'([a-zA-Z]+)')

		subset_combined_results_df = calculate_rank_stability(subset_combined_results_df, rank_columns)

		# Normalize rank and rank stability
		subset_combined_results_df['normalized_rank'] = subset_combined_results_df['combined_final_wavelet_rank'] / subset_combined_results_df['combined_final_wavelet_rank'].max()
		subset_combined_results_df['normalized_stability'] = 1 - subset_combined_results_df['rank_stability']  # Penalize instability

		# Define weights for rank and stability
		alpha = 0.5  # Weight for rank
		beta = 0.5   # Weight for stability

		# Compute composite score
		subset_combined_results_df['composite_score'] = (
			alpha * subset_combined_results_df['normalized_rank'] + 
			beta * subset_combined_results_df['normalized_stability']
		)

		# Add rank bins to the data
		subset_combined_results_df['rank_bin'] = pd.cut(
			subset_combined_results_df['combined_final_wavelet_rank'],
			bins=[0, 10, 20, 50, 100, subset_combined_results_df['combined_final_wavelet_rank'].max()],
			labels=['Top 10', 'Top 20', 'Top 50', 'Top 100', 'Beyond 100']
		)

		# Add unique htid count and stability metrics to the summary
		rank_bin_summary = subset_combined_results_df.groupby(['wavelet_family', 'rank_bin']).agg(
			count=('combined_final_wavelet_rank', 'count'),
			unique_htid=('htid', 'nunique'),  # Count of unique volumes (htid)
			mean_rank_stability=('rank_stability', 'mean'),  # Mean rank stability
			std_rank_stability=('rank_stability', 'std')  # Standard deviation of rank stability
		).reset_index()

		# Total count for each rank_bin
		bin_totals = rank_bin_summary.groupby('rank_bin')['count'].sum().reset_index()
		bin_totals.rename(columns={'count': 'total_bin_count'}, inplace=True)
		rank_bin_summary = rank_bin_summary.merge(bin_totals, on='rank_bin')

		# Total unique htid per rank_bin
		bin_totals_htid = rank_bin_summary.groupby('rank_bin')['unique_htid'].sum().reset_index()
		bin_totals_htid.rename(columns={'unique_htid': 'total_bin_unique_htid'}, inplace=True)
		rank_bin_summary = rank_bin_summary.merge(bin_totals_htid, on='rank_bin')

		# Proportion of all wavelets in each bin (relative to total bin count)
		rank_bin_summary['global_proportion'] = rank_bin_summary['count'] / rank_bin_summary['total_bin_count']

		# Proportion of unique htid in each bin relative to total unique htid for that bin
		rank_bin_summary['htid_proportion'] = rank_bin_summary['unique_htid'] / rank_bin_summary['total_bin_unique_htid']
		rank_bin_summary = rank_bin_summary[rank_bin_summary.rank_bin == 'Top 10'].sort_values(by=['global_proportion', 'htid_proportion', 'mean_rank_stability', 'std_rank_stability', 'count', 'unique_htid'], ascending=[False, False, False, True, False, False])
		top_wavelet_family = rank_bin_summary.iloc[0].wavelet_family
		finalized_subset_combined_results_df = subset_combined_results_df.merge(rank_bin_summary, on=['wavelet_family', 'rank_bin'], how='left')
		volume_dfs.append(finalized_subset_combined_results_df)


In [93]:
# Combine all volume data for the title into one DataFrame
combined_volume_df = pd.concat(volume_dfs, ignore_index=True)

# Normalize rank and stability across all volumes
combined_volume_df['final_normalized_rank'] = combined_volume_df['combined_final_wavelet_rank'] / combined_volume_df['combined_final_wavelet_rank'].max()
combined_volume_df['normalized_stability'] = 1 - combined_volume_df['rank_stability']

# Compute composite score across all volumes
alpha = 0.5  # Weight for rank
beta = 0.5   # Weight for stability
combined_volume_df['final_composite_score'] = (
    alpha * combined_volume_df['final_normalized_rank'] +
    beta * combined_volume_df['normalized_stability']
)

In [94]:
# Aggregate metrics for wavelet families across all volumes
wavelet_summary = combined_volume_df.groupby('wavelet_family').agg(
    mean_composite_score=('final_composite_score', 'mean'),
    mean_rank_stability=('rank_stability', 'mean'),
    std_rank_stability=('rank_stability', 'std'),
    mean_rank=('combined_final_wavelet_rank', 'mean'),
    total_count=('htid', 'count')  # Total number of volumes where this wavelet appears
).reset_index()

# Sort by composite score and rank stability
wavelet_summary = wavelet_summary.sort_values(
    by=['mean_composite_score', 'mean_rank_stability', 'mean_rank'],
    ascending=[True, False, True]
)

In [95]:
# Step 1: Compute normalized metrics
wavelet_summary['normalized_mean_composite_score'] = wavelet_summary['mean_composite_score'] / wavelet_summary['mean_composite_score'].max()
wavelet_summary['normalized_mean_rank_stability'] = wavelet_summary['mean_rank_stability'] / wavelet_summary['mean_rank_stability'].max()
wavelet_summary['normalized_mean_rank'] = 1 - (wavelet_summary['mean_rank'] / wavelet_summary['mean_rank'].max())
wavelet_summary['normalized_total_count'] = wavelet_summary['total_count'] / wavelet_summary['total_count'].max()

# Step 2: Define weights
alpha = 0.4  # Weight for mean composite score
beta = 0.3   # Weight for rank stability
gamma = 0.2  # Weight for mean rank
delta = 0.1  # Weight for total count

# Step 3: Compute final composite score
wavelet_summary['final_wavelet_composite_score'] = (
    alpha * wavelet_summary['normalized_mean_composite_score'] +
    beta * wavelet_summary['normalized_mean_rank_stability'] +
    gamma * wavelet_summary['normalized_mean_rank'] +
    delta * wavelet_summary['normalized_total_count']
)

# Step 4: Sort wavelets by the new composite score
wavelet_summary = wavelet_summary.sort_values(
    by='final_wavelet_composite_score', ascending=False
)

# Step 5: Select the best wavelet family
top_wavelet_family = wavelet_summary.iloc[0]
print(f"Best wavelet family for the title: {top_wavelet_family.wavelet_family}")
wavelet_summary

Best wavelet family for the title: db


,wavelet_family,mean_composite_score,mean_rank_stability,std_rank_stability,mean_rank,total_count,normalized_mean_composite_score,normalized_mean_rank_stability,normalized_mean_rank,normalized_total_count,final_wavelet_composite_score
2,db,0.257046,0.948131,0.023043,111.857828,2293,0.590753,0.984615,0.412991,1.000000,0.714284
9,sym,0.399820,0.742920,0.154795,131.299456,1102,0.918881,0.771508,0.310965,0.480593,0.709257
7,morl,0.435116,0.917188,0.015203,190.555556,36,1.000000,0.952482,0.000000,0.015700,0.687315
4,gaus,0.375286,0.929268,0.031042,164.521311,305,0.862497,0.965027,0.136623,0.133014,0.675133
6,mexh,0.401660,0.925789,0.025829,176.444444,36,0.923112,0.961414,0.074052,0.015700,0.674049
3,dmey,0.396142,0.697527,0.190161,118.534483,58,0.910430,0.724369,0.377953,0.025294,0.659603
1,coif,0.238138,0.952486,0.024024,103.760508,1023,0.547299,0.989138,0.455484,0.446140,0.651372
0,bior,0.216783,0.927746,0.107518,87.437439,1031,0.498219,0.963446,0.541145,0.449629,0.641513
8,rbio,0.195750,0.923997,0.099928,76.350215,1165,0.449880,0.959553,0.599328,0.508068,0.638490
5,haar,0.120158,0.962945,0.012385,49.189189,111,0.276152,1.000000,0.741864,0.048408,0.563674


In [96]:
wavelet_summary.to_clipboard(index=False)

In [97]:
# Add rank bins to the data
combined_volume_df ['rank_bin'] = pd.cut(
    combined_volume_df ['combined_final_wavelet_rank'],
    bins=[0, 10, 20, 50, 100, combined_volume_df ['combined_final_wavelet_rank'].max()],
    labels=['Top 10', 'Top 20', 'Top 50', 'Top 100', 'Beyond 100']
)

# Add unique htid count and stability metrics to the summary
final_rank_bin_summary = combined_volume_df .groupby(['wavelet_family', 'rank_bin']).agg(
    count=('combined_final_wavelet_rank', 'count'),
    unique_htid=('htid', 'nunique'),  # Count of unique volumes (htid)
    mean_rank_stability=('rank_stability', 'mean'),  # Mean rank stability
    std_rank_stability=('rank_stability', 'std')  # Standard deviation of rank stability
).reset_index()

# Total count for each rank_bin
final_bin_totals = final_rank_bin_summary.groupby('rank_bin')['count'].sum().reset_index()
final_bin_totals.rename(columns={'count': 'total_bin_count'}, inplace=True)
final_rank_bin_summary = final_rank_bin_summary.merge(final_bin_totals, on='rank_bin')

# Total unique htid per rank_bin
final_bin_totals_htid = final_rank_bin_summary.groupby('rank_bin')['unique_htid'].sum().reset_index()
final_bin_totals_htid.rename(columns={'unique_htid': 'total_bin_unique_htid'}, inplace=True)
final_rank_bin_summary = final_rank_bin_summary.merge(final_bin_totals_htid, on='rank_bin')

# Proportion of all wavelets in each bin (relative to total bin count)
final_rank_bin_summary['global_proportion'] = final_rank_bin_summary['count'] / final_rank_bin_summary['total_bin_count']

# Proportion of unique htid in each bin relative to total unique htid for that bin
final_rank_bin_summary['htid_proportion'] = final_rank_bin_summary['unique_htid'] / final_rank_bin_summary['total_bin_unique_htid']

# Sort order for consistent visualization
sort_order = ['Top 10', 'Top 20', 'Top 50', 'Top 100', 'Beyond 100']

selection = alt.selection_multi(fields=['wavelet_family'], bind='legend')
# Create a bar chart to include rank stability metrics
global_chart = alt.Chart(final_rank_bin_summary).mark_bar().encode(
    x=alt.X('rank_bin:N', title='Rank Bin', sort=sort_order),
    y=alt.Y('global_proportion:Q', title='Proportion of All Volumes (htid)', stack='normalize'),
    color=alt.Color('wavelet_family:N', title='Wavelet Family', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        'wavelet_family',
        'rank_bin',
        'count',
        'global_proportion',
        'unique_htid',
        'htid_proportion',
        'mean_rank_stability',
        'std_rank_stability'
    ],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin (All Volumes)',
    width=300,
    height=300
)

htid_chart = alt.Chart(final_rank_bin_summary).mark_bar().encode(
    x=alt.X('rank_bin:N', title='Rank Bin', sort=sort_order),
    y=alt.Y('htid_proportion:Q', title='Proportion of Unique Volumes (htid)', stack='normalize'),
    color=alt.Color('wavelet_family:N', title='Wavelet Family', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        'wavelet_family',
        'rank_bin',
        'count',
        'global_proportion',
        'unique_htid',
        'htid_proportion',
        'mean_rank_stability',
        'std_rank_stability'
    ],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin (Unique Volumes)',
    width=300,
    height=300
)

# Scatter plot for rank stability metrics
htid_global_chart = alt.Chart(final_rank_bin_summary).mark_point(filled=True).encode(
    x='htid_proportion:Q',
    y='global_proportion:Q',
    color='wavelet_family:N',
    tooltip=[
        'wavelet_family', 
        'rank_bin', 
        'count', 
        'global_proportion', 
        'unique_htid', 
        'htid_proportion', 
        'mean_rank_stability', 
        'std_rank_stability'
    ],
    shape=alt.Shape('rank_bin:N', sort=sort_order),
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Proportion of Wavelets by Rank Bin with Stability Metrics',
    width=300,
    height=300
)

stability_chart = alt.Chart(final_rank_bin_summary).mark_point(filled=True).encode(
	y='mean_rank_stability:Q',
	x='std_rank_stability:Q',
	color='wavelet_family:N',
	tooltip=[
		'wavelet_family', 
		'rank_bin',
		'count',
		'global_proportion',
		'unique_htid',
		'htid_proportion',
		'mean_rank_stability',
		'std_rank_stability'
	],
	opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
	shape=alt.Shape('rank_bin:N', sort=sort_order)
).add_params(selection).properties(
	title='Rank Stability Metrics by Wavelet Family',
	width=300,
	height=300
)

# Combine charts
alt.vconcat(alt.hconcat(global_chart, htid_chart), alt.hconcat(htid_global_chart, stability_chart))

alt.VConcatChart(...)

In [98]:
final_rank_bin_summary[final_rank_bin_summary.rank_bin=='Top 10'].sort_values(by=['global_proportion', 'htid_proportion', 'mean_rank_stability', 'std_rank_stability', 'count', 'unique_htid'], ascending=[False, False, False, True, False, False])

,wavelet_family,rank_bin,count,unique_htid,mean_rank_stability,std_rank_stability,total_bin_count,total_bin_unique_htid,global_proportion,htid_proportion
40,rbio,Top 10,118,10,0.971968,0.012130,360,45,0.327778,0.222222
5,coif,Top 10,91,6,0.989951,0.023222,360,45,0.252778,0.133333
10,db,Top 10,43,9,0.996030,0.008610,360,45,0.119444,0.200000
0,bior,Top 10,39,6,0.998716,0.001512,360,45,0.108333,0.133333
20,gaus,Top 10,29,4,0.999768,0.000641,360,45,0.080556,0.088889
45,sym,Top 10,28,5,0.767999,0.189763,360,45,0.077778,0.111111
25,haar,Top 10,9,3,0.999627,0.000367,360,45,0.025000,0.066667
15,dmey,Top 10,2,1,0.520729,0.000000,360,45,0.005556,0.022222
30,mexh,Top 10,1,1,0.996177,NaN,360,45,0.002778,0.022222
35,morl,Top 10,0,0,NaN,NaN,360,45,0.000000,0.000000


In [99]:
# Step 1: Merge rank bin summary into wavelet summary
rank_bin_summary_top10 = final_rank_bin_summary[final_rank_bin_summary.rank_bin == 'Top 10'][[
    'wavelet_family', 'global_proportion', 'htid_proportion'
]]

# Merge with the wavelet summary DataFrame
wavelet_summary = wavelet_summary.merge(
    rank_bin_summary_top10, on='wavelet_family', how='left'
).fillna(0)  # Fill NaNs with 0 for wavelets that don't appear in the Top 10 bin

# Step 2: Normalize all relevant metrics
wavelet_summary['normalized_mean_composite_score'] = wavelet_summary['mean_composite_score'] / wavelet_summary['mean_composite_score'].max()
wavelet_summary['normalized_mean_rank_stability'] = wavelet_summary['mean_rank_stability'] / wavelet_summary['mean_rank_stability'].max()
wavelet_summary['normalized_mean_rank'] = 1 - (wavelet_summary['mean_rank'] / wavelet_summary['mean_rank'].max())
wavelet_summary['normalized_total_count'] = wavelet_summary['total_count'] / wavelet_summary['total_count'].max()
wavelet_summary['normalized_global_proportion'] = wavelet_summary['global_proportion'] / wavelet_summary['global_proportion'].max()
wavelet_summary['normalized_htid_proportion'] = wavelet_summary['htid_proportion'] / wavelet_summary['htid_proportion'].max()

# Step 3: Define weights for all metrics
alpha = 0.3  # Weight for mean composite score
beta = 0.25  # Weight for rank stability
gamma = 0.15  # Weight for rank
delta = 0.1   # Weight for total count
epsilon = 0.1  # Weight for global proportion
zeta = 0.1    # Weight for HTID proportion

# Step 4: Compute the final composite score
wavelet_summary['final_wavelet_composite_score'] = (
    alpha * wavelet_summary['normalized_mean_composite_score'] +
    beta * wavelet_summary['normalized_mean_rank_stability'] +
    gamma * wavelet_summary['normalized_mean_rank'] +
    delta * wavelet_summary['normalized_total_count'] +
    epsilon * wavelet_summary['normalized_global_proportion'] +
    zeta * wavelet_summary['normalized_htid_proportion']
)

# Step 5: Sort wavelets by the final composite score
wavelet_summary = wavelet_summary.sort_values(
    by='final_wavelet_composite_score', ascending=False
)

# Step 6: Select the best wavelet family
top_wavelet_family = wavelet_summary.iloc[0]
print(f"Best wavelet family for the title: {top_wavelet_family.wavelet_family}")
wavelet_summary

Best wavelet family for the title: rbio


,wavelet_family,mean_composite_score,mean_rank_stability,std_rank_stability,mean_rank,total_count,normalized_mean_composite_score,normalized_mean_rank_stability,normalized_mean_rank,normalized_total_count,final_wavelet_composite_score,global_proportion,htid_proportion,normalized_global_proportion,normalized_htid_proportion
8,rbio,0.195750,0.923997,0.099928,76.350215,1165,0.449880,0.959553,0.599328,0.508068,0.715558,0.327778,0.222222,1.000000,1.0
0,db,0.257046,0.948131,0.023043,111.857828,2293,0.590753,0.984615,0.412991,1.000000,0.711769,0.119444,0.200000,0.364407,0.9
6,coif,0.238138,0.952486,0.024024,103.760508,1023,0.547299,0.989138,0.455484,0.446140,0.661530,0.252778,0.133333,0.771186,0.6
1,sym,0.399820,0.742920,0.154795,131.299456,1102,0.918881,0.771508,0.310965,0.480593,0.636974,0.077778,0.111111,0.237288,0.5
7,bior,0.216783,0.927746,0.107518,87.437439,1031,0.498219,0.963446,0.541145,0.449629,0.609513,0.108333,0.133333,0.330508,0.6
3,gaus,0.375286,0.929268,0.031042,164.521311,305,0.862497,0.965027,0.136623,0.133014,0.598377,0.080556,0.088889,0.245763,0.4
4,mexh,0.401660,0.925789,0.025829,176.444444,36,0.923112,0.961414,0.074052,0.015700,0.540812,0.002778,0.022222,0.008475,0.1
2,morl,0.435116,0.917188,0.015203,190.555556,36,1.000000,0.952482,0.000000,0.015700,0.539691,0.000000,0.000000,0.000000,0.0
5,dmey,0.396142,0.697527,0.190161,118.534483,58,0.910430,0.724369,0.377953,0.025294,0.525138,0.005556,0.022222,0.016949,0.1
9,haar,0.120158,0.962945,0.012385,49.189189,111,0.276152,1.000000,0.741864,0.048408,0.486593,0.025000,0.066667,0.076271,0.3
